# CARE Registration Notebook

This notebook handles image registration for CARE (Content-Aware Image Restoration) pipeline.

## Setup

Make sure you have the required packages installed:
```bash
pip install SimpleITK tifffile matplotlib aicsimageio aicspylibczi scipy numpy
```

In [1]:
import numpy as np
import SimpleITK as sitk
import matplotlib.pyplot as plt
from tifffile import imread, imwrite
import dask.array as da
from aicsimageio import AICSImage

# Configure matplotlib for inline display
%matplotlib inline

### Functions

In [4]:
from scipy.ndimage import zoom

def downsample_volume(volume, z_factor=0.5, xy_factor=0.5, order=1):
    """
    Downsample a 3D volume (Z, Y, X) by given factors.

    Parameters
    ----------
    volume : np.ndarray
        3D image (Z, Y, X) to downsample.
    z_factor : float
        Scaling factor along Z (e.g. 0.5 = half as many slices).
    xy_factor : float
        Scaling factor along Y and X (e.g. 0.25 = 4× smaller laterally).
    order : int
        Interpolation order: 0=nearest, 1=linear, 3=cubic.

    Returns
    -------
    small : np.ndarray
        Downsampled 3D volume.
    """
    assert volume.ndim == 3, "Input must be 3D (Z, Y, X)"
    zoom_factors = (z_factor, xy_factor, xy_factor)
    small = zoom(volume, zoom_factors, order=order)
    return small.astype(np.float32)


def load_czi(path):
    """Load and normalize CZI image file."""
    # Load image with AICSImageIO
    img = AICSImage(path)

    data = img.get_image_data()

    # typical shape: (1,1,1,Z,Y,X,1)
    data = np.squeeze(data)
    # normalize 0–1
    data = data.astype(np.float32)
    data = (data - data.min()) / (data.max() - data.min() + 1e-8)
    return data

### Configuration and Execution

**Note:** Update the file paths below to match your local directory structure.

In [5]:
import os

# === 1. Define file paths ===
# TODO: Update these paths to match your local directory structure
fixed_path  = r"/Volumes/WDRIVE/Group Fritzsche/Ed/lattice/08102025_fixed_LN_CARE/CARE_data/raw/2ii4_lifeactGFP_50pc_1ms-01.czi"
moving_path = r"/Volumes/WDRIVE/Group Fritzsche/Ed/lattice/08102025_fixed_LN_CARE/CARE_data/raw/2ii4_lifeactGFP_50pc_50ms-01.czi"

dir_path = os.path.dirname(moving_path)
name     = os.path.basename(moving_path[:-4])
print(dir_path)

fixed_np_full  = load_czi(fixed_path)
print(fixed_np_full.dtype)
print(fixed_np_full.shape)

# Create output directory if it doesn't exist
output_dir = os.path.join(dir_path, "registered")
os.makedirs(output_dir, exist_ok=True)

imwrite(os.path.join(output_dir, name + "_5ms_fixed.tif"), fixed_np_full.astype(np.float32))
print("💾 Saved as: fixed.tif")

moving_np_full = load_czi(moving_path)

z_factor  = 0.5
xy_factor = 0.5
fixed_np_down  = downsample_volume(fixed_np_full , z_factor=z_factor, xy_factor=xy_factor, order=1)
moving_np_down = downsample_volume(moving_np_full, z_factor=z_factor, xy_factor=xy_factor, order=1)

# Convert numpy arrays to SimpleITK images
fixed_sitk_full  = sitk.GetImageFromArray(fixed_np_full)
moving_sitk_full = sitk.GetImageFromArray(moving_np_full)
fixed_sitk_down  = sitk.GetImageFromArray(fixed_np_down)
moving_sitk_down = sitk.GetImageFromArray(moving_np_down)

# Create a plain 3D translation transform
initial_tx = sitk.TranslationTransform(3)

# Set up registration
reg = sitk.ImageRegistrationMethod()
reg.SetMetricAsCorrelation()
reg.SetInterpolator(sitk.sitkLinear)
reg.SetOptimizerAsRegularStepGradientDescent(
    learningRate=1.0, minStep=1e-4, numberOfIterations=200
)
reg.SetInitialTransform(initial_tx, inPlace=False)

print("🔄 Running translation-only registration...")
final_tx = reg.Execute(fixed_sitk_down, moving_sitk_down)
dz, dy, dx = final_tx.GetParameters()
full_translation = [dz * 1/z_factor, dy * 1/xy_factor, dx * 1/xy_factor]
print("✅ Translation (Z, Y, X):", full_translation)
tx_full = sitk.TranslationTransform(3)
tx_full.SetParameters(full_translation)


# Apply transform
registered_sitk = sitk.Resample(moving_sitk_full, fixed_sitk_full, tx_full, sitk.sitkLinear, 0.0, moving_sitk_full.GetPixelID())
registered_np = sitk.GetArrayFromImage(registered_sitk)
print(registered_np.shape)

imwrite(os.path.join(output_dir, name + "_5ms_registered.tif"), registered_np.astype(np.float32))
print("💾 Saved as: pos_registered_translation.tif")

# === 5. Quick visual check (middle slice overlay) ===
z = fixed_np_full.shape[0] // 2
overlay = np.stack([fixed_np_full[z], registered_np[z], np.zeros_like(fixed_np_full[z])], axis=-1)

plt.figure(figsize=(10,4))
plt.subplot(1,3,1); plt.imshow(fixed_np_full[z], cmap='gray'); plt.title("Fixed"); plt.axis('off')
plt.subplot(1,3,2); plt.imshow(registered_np[z], cmap='gray'); plt.title("Registered"); plt.axis('off')
plt.subplot(1,3,3); plt.imshow(overlay); plt.title("Overlay"); plt.axis('off')
plt.show()

/Volumes/WDRIVE/Group Fritzsche/Ed/lattice/08102025_fixed_LN_CARE/CARE_data/raw


KeyboardInterrupt: 